# **Compare Topics between Twitter and TikTok descriptions for the same brand: Gucci**

In this notebook I will do NLP of Gucci TikTok Descriptions and Twitter tweets. More scpecially LDA topic modelling:

https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05

https://lazarinastoy.com/topic-modelling-lda/

https://radimrehurek.com/gensim/models/ldamodel.html

Code-along video: https://www.youtube.com/watch?v=TKjjlp5_r7o


## **Installing and importing the librairies**

In [ ]:
%%capture

# Install all needed libraries

!pip install pyLDAvis==2.1.2 -qq
!pip install gensim -qq

In [ ]:
# import necessary libraries
import pandas as pd
from datetime import datetime
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


## **Importing and Process TikTok Json Data**

In [ ]:
# Load TikTok Data
df = pd.read_json('/content/drive/MyDrive/Final Project/4. Natural Language Processing/GucciTikTokDesc.json')
df.head()

,id,desc,createTime
0,6906535387636780032,Moving with the #GucciGrip watch,1608053082
1,6904284575254531072,Get it with #GucciGift,1607529023
2,6904224653649153024,Let’s do this one together. #GucciGift,1607515072
3,6903903579703086080,Moving into the holidays. #GucciGift,1607440316
4,6903809411899264000,Dancing at the Gucci Party. #GucciGift,1607418391


In [ ]:
# Inspect data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          254 non-null    int64 
 1   desc        254 non-null    object
 2   createTime  254 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ KB


In [ ]:
# Remove duplicates
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 171
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          142 non-null    int64 
 1   desc        142 non-null    object
 2   createTime  142 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ KB


In [ ]:
# Reformat timestamp dates
df['createTime'] = [datetime.fromtimestamp(x) for x in df['createTime']]
df['createTime'] = df['createTime'].dt.date
df.head()

,id,desc,createTime
0,6906535387636780032,Moving with the #GucciGrip watch,2020-12-15
1,6904284575254531072,Get it with #GucciGift,2020-12-09
2,6904224653649153024,Let’s do this one together. #GucciGift,2020-12-09
3,6903903579703086080,Moving into the holidays. #GucciGift,2020-12-08
4,6903809411899264000,Dancing at the Gucci Party. #GucciGift,2020-12-08


In [ ]:
# Print the time period we will have to apply to twitter data to analyze the same scope
date_min = df['createTime'].min()
date_max = df['createTime'].max()

print(date_min)
print(date_max)

2020-02-07
2021-10-30


## **Preparing TikTok Data**


In [ ]:
# Set stopwords
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Lemmatize TikTok Descriptions
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(df['desc'])
print(lemmatized_texts[0][0:90])

move watch


In [ ]:
# Gensim processing of TikTok descriptions
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])

['move', 'watch']


In [ ]:
# Set the corpus of words
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print(corpus[0][0:20])

word = id2word[[0][:1][0]]
print(word)

[(0, 1), (1, 1)]
move


In [ ]:
# Create LDA Model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

## **Vizualizing TikTok Data**

In [ ]:
# TikTok Topic Modeling Visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.290814 -0.159959       1        1  13.088749
17     0.442194 -0.046931       2        1   8.114436
20    -0.404487 -0.006676       3        1   6.749225
24    -0.251870 -0.295061       4        1   4.488346
7      0.022038 -0.448702       5        1   4.479424
9     -0.081550 -0.246076       6        1   4.147577
5     -0.114620 -0.371192       7        1   4.088272
3      0.326072  0.140896       8        1   4.079278
22     0.116405  0.351223       9        1   3.946412
15     0.174268 -0.365603      10        1   3.692813
12     0.266220 -0.258293      11        1   3.591178
29     0.160707  0.236593      12        1   3.323721
19     0.290742  0.233596      13        1   3.251740
21    -0.269345  0.260042      14        1   2.581469
27    -0.053415  0.375760      15        1   2.559369
6     -0.135620  0.331666      16        1   2.401104
1     -0.202078  0.116033      17        1   2.342581
8     -0.317290  0.049856      18        1   2.282905
11    -0.034559 -0.146910      19        1   2.273903
13    -0.296773 -0.056278      20        1   2.254330
23     0.278702  0.065175      21        1   2.151318
2     -0.014851  0.239551      22        1   2.010284
14     0.213799  0.088366      23        1   1.983589
0     -0.242331 -0.181610      24        1   1.958954
4     -0.064595  0.092653      25        1   1.782348
25     0.058351 -0.143857      26        1   1.730162
16    -0.146093  0.195336      27        1   1.581816
18     0.101805 -0.045822      28        1   1.231677
28    -0.144577  0.001263      29        1   1.013197
26     0.031939 -0.005039      30        1   0.819821, topic_info=              Term       Freq      Total Category  logprob  loglift
51        lipstick  16.000000  16.000000  Default  30.0000  30.0000
24             new  34.000000  34.000000  Default  29.0000  29.0000
45            daze  12.000000  12.000000  Default  28.0000  28.0000
46            life  12.000000  12.000000  Default  27.0000  27.0000
35           video  21.000000  21.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
37           speak   0.019180   4.240850  Topic30  -6.0766  -0.5948
25          series   0.019180  16.696497  Topic30  -6.0766  -1.9653
176            day   0.019179   1.376790  Topic30  -6.0767   0.5301
184  craftsmanship   0.019179   1.376790  Topic30  -6.0767   0.5301
183         boogie   0.019179   1.376790  Topic30  -6.0767   0.5301

[1504 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
133      14  0.343222     accidentally
133      22  0.343222     accidentally
133      27  0.343222     accidentally
220       1  0.726327           animal
39        3  0.878939         announce
...     ...       ...              ...
47       11  0.732282  winston__nguyen
8        28  0.793816             wipe
114       2  0.905777            woman
79       20  0.820792             work
166       1  0.648229               ºð

[336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 18, 21, 25, 8, 10, 6, 4, 23, 16, 13, 30, 20, 22, 28, 7, 2, 9, 12, 14, 24, 3, 15, 1, 5, 26, 17, 19, 29, 27])

## **Compare with Tweet Data**

In [ ]:
# Load and get only Gucci and McQueen Tweets
dftweets = pd.read_csv('/content/drive/MyDrive/Final Project/2. Sentiment Analysis/Tweet_Posts.csv')

dftweets = dftweets[dftweets['brand']=='Gucci']

# only the relevent columns
dftweets = dftweets.iloc[:, [3,4,5]]

# replace nan 
dftweets.replace(np.nan,'',inplace=True)

# show data
dftweets.head(3)

,id,created_at,text
39217,1458706068046065664,2021-11-11 08:00:02,minashin joins as new gucci global brand ambas...
39218,1458675864183484419,2021-11-11 06:00:01,joining gucci as new global brand ambassador i...
39219,1458226659363106816,2021-11-10 00:15:02,at the london premiere of and pinault all wo...


In [ ]:
# Reformat time variable
dftweets['created_at'] = pd.to_datetime(dftweets['created_at'])
dftweets['created_at'] = dftweets['created_at'].dt.date
dftweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3250 entries, 39217 to 42466
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3250 non-null   int64 
 1   created_at  3250 non-null   object
 2   text        3250 non-null   object
dtypes: int64(1), object(2)
memory usage: 101.6+ KB


In [ ]:
# Filter data between the two dates of the scope
dftweets = dftweets.loc[(dftweets['created_at'] >= date_min)
                     & (dftweets['created_at'] <= date_max)]

dftweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601 entries, 39282 to 40882
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1601 non-null   int64 
 1   created_at  1601 non-null   object
 2   text        1601 non-null   object
dtypes: int64(1), object(2)
memory usage: 50.0+ KB


In [ ]:
# Lemmatize tweets data
lemmatized_tweets = lemmatization(dftweets['text'])
print(lemmatized_tweets[0][0:90])

come soon boulevard city stud


In [ ]:
# Gensim processing tweets data
tweets_words = gen_words(lemmatized_tweets)
print(tweets_words[0][0:20])

['come', 'soon', 'boulevard', 'city', 'stud']


In [ ]:
# Set corpus of words
id2word_tweets = corpora.Dictionary(tweets_words)

corpus_tweets = []
for text in tweets_words:
    new = id2word_tweets.doc2bow(text)
    corpus_tweets.append(new)

print(corpus_tweets[0][0:20])

word_tweets = id2word_tweets[[0][:1][0]]
print(word_tweets)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
boulevard


In [ ]:
# Create LDA model
lda_model_tweets = gensim.models.ldamodel.LdaModel(corpus=corpus_tweets,
                                           id2word=id2word_tweets,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
# Tweets Topic Modeling visualization
pyLDAvis.enable_notebook()
vis_tweets = pyLDAvis.gensim.prepare(lda_model_tweets, corpus_tweets, id2word_tweets, mds="mmds", R=30)
vis_tweets

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.437679  0.171696       1        1  7.701775
23     0.147851 -0.415871       2        1  7.281931
18     0.408045  0.206615       3        1  7.204098
6      0.267167  0.303331       4        1  4.082282
28     0.265074 -0.116390       5        1  3.654993
10    -0.019177 -0.377717       6        1  3.597169
11    -0.208000  0.335727       7        1  3.566699
13     0.369125  0.015211       8        1  3.540615
8     -0.317640 -0.187932       9        1  3.526995
24    -0.447217 -0.085958      10        1  3.458120
12    -0.210580  0.215313      11        1  3.422039
26    -0.133811 -0.355678      12        1  3.279320
22    -0.238154 -0.291220      13        1  3.254777
7      0.296955  0.007762      14        1  3.239018
29     0.061341  0.481912      15        1  3.155486
0      0.394915 -0.221581      16        1  2.999843
21     0.170448 -0.182917      17        1  2.923206
5     -0.294802  0.129394      18        1  2.875439
3     -0.204681 -0.110849      19        1  2.860517
20     0.088657  0.152634      20        1  2.810404
25     0.110226  0.286055      21        1  2.441804
2     -0.040444  0.205236      22        1  2.418660
4     -0.068662  0.320759      23        1  2.380628
17     0.167471 -0.272431      24        1  2.239715
19     0.196374  0.118306      25        1  2.171216
27    -0.070607 -0.135500      26        1  2.165567
9      0.013632 -0.208590      27        1  2.094032
1     -0.247623  0.001838      28        1  2.037495
16     0.052570 -0.019475      29        1  1.872061
14    -0.070776  0.030321      30        1  1.744097, topic_info=             Term        Freq       Total Category  logprob  loglift
83            new  288.000000  288.000000  Default  30.0000  30.0000
415       perform  195.000000  195.000000  Default  29.0000  29.0000
1721          pay  186.000000  186.000000  Default  28.0000  28.0000
380       tribute  135.000000  135.000000  Default  27.0000  27.0000
2804   davidbowie  133.000000  133.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
1951       ironic    1.193538    2.065436  Topic30  -5.1994   3.5005
2299   ambassador    1.150043    2.021942  Topic30  -5.2365   3.4847
2301         girl    1.150043    2.021942  Topic30  -5.2365   3.4847
2300       father    1.150043    2.021942  Topic30  -5.2365   3.4847
2302  responsibly    1.150043    2.021942  Topic30  -5.2365   3.4847

[995 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2129      8  0.481541     abdicate
2142     28  0.785792         able
1868      7  0.738257        about
2283     21  0.643359     absolute
2630     16  0.601722  accessorize
...     ...       ...          ...
129       8  0.967803         year
1464      7  0.726511       yellow
2532     19  0.795887          yet
2533     19  0.795887         zoom
2225      4  0.900136    zumirosow

[969 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 24, 19, 7, 29, 11, 12, 14, 9, 25, 13, 27, 23, 8, 30, 1, 22, 6, 4, 21, 26, 3, 5, 18, 20, 28, 10, 2, 17, 15])